In [1]:
import warnings
warnings.filterwarnings('ignore')
from tqdm.autonotebook import tqdm
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import gmean
from scipy.stats.mstats import hmean
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel
import statsmodels.api as sm
from itertools import combinations
from pathlib import Path
import pickle
import datetime

In [2]:
#Hyperparamaters
w1 = 100
w2 = 3300
w3 = 3000
w4 = 13000
w5 = 22000
extent1 = 4
extent2 = 3
extent3 = 70
extent4 = 2
extent5 = 130

In [3]:
#Path locations
path_dir = Path('D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/Sessions')
path = 'D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/Sessions'
testval = pd.read_csv(path+'/TestActualData.csv')
spatialt = pd.read_csv(path+'/spatialT.csv')
spatialh = pd.read_csv(path+'/spatialH.csv')
testt = pd.read_csv(path+'/testT.csv')
testh = pd.read_csv(path+'/testH.csv')
with open(path+'/ID2Index', 'rb') as handle:
    ID2Index = pickle.load(handle)
with open(path+'/TestID2Index', 'rb') as handle:
    TestID2Index = pickle.load(handle)

In [4]:
#probabilistic ensemble model function
def MLPred(y, data, spatial, did, id2index, xval, index, extent1, extent2, extent3, extent4, extent5, w1, w2, w3, w4, w5, deg, r):

    #Bayesian Ridge Polynomial Model (Predictor variable) Dataset Configuration
    if index-extent1>=0 and index+extent1+1<=data.shape[0]:
        D1=data.iloc[index-extent1:index+extent1+1,2:4]
    elif index-extent1<=0 and index+extent1+1>=data.shape[0]:
        D1=data.iloc[:,2:4]
    elif index-extent1<=0:
        D1=data.iloc[0:extent1*2+1,2:4]
    else:
        D1=data.iloc[data.shape[0]-extent1*2:data.shape[0],2:4]
    D1.dropna(inplace = True)
    if y=='h':
        X1=D1.iloc[:,0:1].values
        Y1=D1.iloc[:,1].values
    elif y=='t':
        X1=D1.iloc[:,1:2].values
        Y1=D1.iloc[:,0].values
    w1=max(0,w1*(((D1.shape[0]-deg)/(2*extent1-deg))**deg))
    
    #Bayesian Ridge Linear Model (Predictor variable) Dataset Configuration
    if index-extent2>=0 and index+extent2+1<=data.shape[0]:
        D2=data.iloc[index-extent2:index+extent2+1,2:4]
    elif index-extent2<=0 and index+extent2+1>=data.shape[0]:
        D2=data.iloc[:,2:4]
    elif index-extent2<=0:
        D2=data.iloc[0:extent2*2+1,2:4]
    else:
        D2=data.iloc[data.shape[0]-extent2*2:data.shape[0],2:4]
    D2.dropna(inplace = True)
    if D2.shape[0]<=extent2:
        w2=0
    if y=='h':
        X2=D2.iloc[:,0:1].values
        Y2=D2.iloc[:,1].values
    elif y=='t':
        X2=D2.iloc[:,1:2].values
        Y2=D2.iloc[:,0].values
    w2=max(0,w2*(D1.shape[0]-1)/(2*extent1-1))
        
    #Gaussian Process Regression Model (Objective variable - spatial/at other nodes) Dataset Configuration
    sid=id2index[did]
        
    if 1 <= did <= 1367:
        node = 1
    elif 1368 <= did <= 2621:
        node = 2
    elif 2622 <= did <= 3989:
        node = 5
    elif 3990 <= did <= 5352:
        node = 7
    elif 5353 <= did <= 6583:
        node = 8
        
    spatialo=spatial.copy()
    xvec=spatialo.iloc[sid-1:sid+1+1,:]
    xvec.drop(['X'+str(node),'ID'], axis=1, inplace=True)
    xvec.dropna(inplace = True)
    if xvec.shape[0]==0:
        w3=0
    
    adj=0
    if sid-extent3>=0 and sid+extent3+1<=spatialo.shape[0]:
        while abs(spatialo['timestamp'].iloc[sid+extent3-adj]-spatialo['timestamp'].iloc[sid-extent3+adj])>(extent3-adj)*r:
            adj+=1
            if adj==extent3:
                w3=0
            if not w3:
                break
        if w3:
            D3=spatialo.iloc[sid-extent3+adj:sid+extent3-adj+1,:]
    elif sid-extent3<0:
        while abs(spatialo['timestamp'].iloc[extent3*2-adj]-spatialo['timestamp'].iloc[0+adj])>(extent3-adj)*r:
            adj+=1
            if adj==extent3:
                w3=0
            if not w3:
                break
        if w3:
            D3=spatialo.iloc[0+adj:extent3*2-adj+1]
    elif sid+extent3+1>spatialo.shape[0]:
        while abs(spatialo['timestamp'].iloc[spatialo.shape[0]-1-adj]-spatialo['timestamp'].iloc[spatialo.shape[0]-extent3*2+adj])>(extent3-adj)*r:
            adj+=1
            if adj==extent3:
                w3=0
            if not w3:
                break
        if w3:
            D3=spatialo.iloc[spatialo.shape[0]-extent3*2+adj:spatialo.shape[0]-adj,:]
    if w3:
        D3.dropna(inplace = True)
        if D3.shape[0]<5:
            w3=0
        Y3=D3['X'+str(node)]
        D3.drop(['X'+str(node),'ID'], axis=1, inplace=True)
        X3=D3
        w3=max(0,w3*((D3.shape[0]-5)/(2*extent3-5)))

    
    #Bayesian Ridge Polynomial Model (Objective variable - temporal) Dataset Configuration
    sta=datetime.datetime(2014, 1, 1)
    data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S')
    data['timestamp']=data['timestamp']-sta
    data['timestamp'] = (data['timestamp']).apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
    xtime = data.at[index, 'timestamp']
    if index-extent4>=0 and index+extent4+1<=data.shape[0]:
        D4=data.iloc[index-extent4:index+extent4+1,1:4]
    elif index-extent4<=0 and index+extent4+1>=data.shape[0]:
        D4=data.iloc[:,1:4]
    elif index-extent4<=0:
        D4=data.iloc[0:extent4*2+1,1:4]
    else:
        D4=data.iloc[data.shape[0]-extent4*2:data.shape[0],1:4]
    D4.dropna(inplace = True)
    
    if y=='h':
        X4=D4.iloc[:,0:1].values
        Y4=D4.iloc[:,2].values
    elif y=='t':
        X4=D4.iloc[:,0:1].values
        Y4=D4.iloc[:,1].values
    tdeg=1
    if D4.shape[0]==0:
        w4=0
    elif D4.shape[0]>2:
        tdeg=2
    
    
    #Bayesian Ridge Linear Model (Objective variable - spatial/at other nodes) Dataset Configuration
    spatialb= spatial.copy()
    xvec5=spatialb.iloc[sid-1:sid+1+1,:]
    xvec5.drop(['X'+str(node),'ID', 'timestamp'], axis=1, inplace=True)
    xvec5.dropna(inplace = True)
    if xvec5.shape[0]==0:
        w5=0
    
    adj=0
    if sid-extent5>=0 and sid+extent5+1<=spatialb.shape[0]:
        while abs(spatialb['timestamp'].iloc[sid+extent5-adj]-spatialb['timestamp'].iloc[sid-extent5+adj])>(extent5-adj)*r:
            adj+=1
            if adj==extent5:
                w5=0
            if not w5:
                break
        if w5:
            D5=spatialb.iloc[sid-extent5+adj:sid+extent5-adj+1,:]
    elif sid-extent5<0:
        while abs(spatialb['timestamp'].iloc[extent5*2-adj]-spatialb['timestamp'].iloc[0+adj])>(extent5-adj)*r:
            adj+=1
            if adj==extent5:
                w5=0
            if not w5:
                break
        if w5:
            D5=spatialb.iloc[0+adj:extent5*2-adj+1]
    elif sid+extent5+1>spatialb.shape[0]:
        while abs(spatialb['timestamp'].iloc[spatialb.shape[0]-1-adj]-spatialb['timestamp'].iloc[spatialb.shape[0]-extent5*2+adj])>(extent5-adj)*r:
            adj+=1
            if adj==extent5:
                w5=0
            if not w5:
                break
        if w5:
            D5=spatialb.iloc[spatialb.shape[0]-extent5*2+adj:spatialb.shape[0]-adj,:]
    if w5:
        D5.dropna(inplace = True)
        if D5.shape[0]<5:
            w5=0
        Y5=D5['X'+str(node)]
        D5.drop(['X'+str(node),'ID', 'timestamp'], axis=1, inplace=True)
        X5=D5
        w5=max(0,w5*((D5.shape[0]-5)/(2*extent5-5)))
    
    #Model Definitions
    if w1:
        poly_reg=PolynomialFeatures(degree=deg,include_bias=False)
        X_poly=poly_reg.fit_transform(X1)
        poly_reg.fit(X_poly,Y1)
        ML_reg=BayesianRidge()
        ML_reg.fit(X_poly,Y1)
        polypred=ML_reg.predict(poly_reg.fit_transform([[xval]]))
        Y_p=polypred[0]
    else:
        Y_p=0
        
    if w2:
        LR_reg=BayesianRidge()
        LR_reg.fit(X2,Y2)
        lrpred=LR_reg.predict([[xval]])
        Y_l=lrpred[0]
    else:
        Y_l=0
        
    if w3:
        kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
        LO_reg=GaussianProcessRegressor(kernel=kernel)
        LO_reg.fit(X3,Y3)
        lopred=LO_reg.predict(xvec)
        Y_o=lopred.mean()
    else:
        Y_o=0
        
    if w4:
        if D4.shape[0]==1:
            Y_t=Y4.mean()
        else:
            poly_reg4=PolynomialFeatures(degree=tdeg,include_bias=False)
            X_poly4=poly_reg4.fit_transform(X4)
            poly_reg4.fit(X_poly4,Y4)
            ML_reg4=BayesianRidge()
            ML_reg4.fit(X_poly4,Y4)
            polypred4=ML_reg4.predict(poly_reg4.fit_transform([[xtime]]))
            Y_t=polypred4[0]
    else:
        Y_t=0
        
    if w5:
        LB_reg=BayesianRidge()
        LB_reg.fit(X5,Y5)
        lbpred=LB_reg.predict(xvec5)
        Y_b=lbpred.mean()
    else:
        Y_b=0
    
    #Absurd value detection algorithm
    if y == 't':
        MAXPOSVALUE=38
        MINPOSVALUE=8
    elif y == 'h':
        MAXPOSVALUE=78
        MINPOSVALUE=-26
        
    if MINPOSVALUE>Y_p or Y_p>MAXPOSVALUE:
        w1=0
        Y_p=0
    if MINPOSVALUE>Y_l or Y_l>MAXPOSVALUE:
        w2=0
        Y_l=0
    if MINPOSVALUE>Y_o or Y_o>MAXPOSVALUE:
        w3=0
        Y_o=0
    if MINPOSVALUE>Y_t or Y_t>MAXPOSVALUE:
        w4=0
        Y_t=0
    if MINPOSVALUE>Y_b or Y_b>MAXPOSVALUE:
        w5=0
        Y_b=0
    
    if abs(Y_p)<=0.001:
        w1=0
    if abs(Y_l)<=0.001:
        w2=0
    if abs(Y_o)<=0.001:
        w3=0
    if abs(Y_b)<=0.001:
        w5=0
    
    
    ylist = [i for i in [Y_p, Y_l, Y_o, Y_t, Y_b] if i != 0]
    stdlist=[]
    clist=[]
    for c in ylist:
        chk1=np.mean([abs(i[0]-i[1]) for i in list(combinations([i for i in ylist if i != c],2))])
        chk2=np.mean([abs(i-c) for i in ylist if i != c])
        stdlist.append(abs(chk1-chk2))
        clist.append(c)
    for i in range(len(clist)):
        if len(stdlist)>2 and stdlist[i]==max(stdlist) and stdlist[i]>=2*np.mean([j for j in stdlist if j != stdlist[i]]):
            c = clist[i]
            if Y_p==c:
                w1=0
            elif Y_l==c:
                w2=0
            elif Y_o==c:
                w3=0
            elif Y_t==c:
                w4=0
            elif Y_b==c:
                w5=0
    return (w1*Y_p+w2*Y_l+w3*Y_o+w4*Y_t+w5*Y_b)/(w1+w2+w3+w4+w5)

In [5]:
#Model Testing and Prediction
polydeg = 2
risk_appetite = 4.4
pred=pd.DataFrame(columns=['ID','Predictions'])
test=pd.DataFrame(columns=['ID','Predictions'])
for k in tqdm([1,2,5,7,8]):
    for l in tqdm([*range(1,9)], leave=False):
        dfo=pd.read_csv(path+'/PredSO_N50'+str(k)+'_'+str(l).zfill(2)+'.csv')
        df=pd.read_csv(path+'/PredS_N50'+str(k)+'_'+str(l).zfill(2)+'.csv')
        df.reset_index(drop=True, inplace=True)
        dfo.reset_index(drop=True, inplace=True)
        df2=df.copy()
        for index in tqdm(range(dfo.shape[0]), leave=False):
            if np.isnan(dfo.iloc[index]['temperature']):
                yp=MLPred('t', df2, spatialt, dfo.iloc[index]['ID'], ID2Index, dfo.iloc[index]['humidity'], index, extent1, extent2, extent3, extent4, extent5, w1, w2, w3, w4, w5, polydeg, risk_appetite)
                new_row = pd.Series([dfo.iloc[index]['ID'], yp], index=pred.columns)
                pred = pred.append(new_row,ignore_index=True)
                df2['timestamp']=df['timestamp']
                
            if np.isnan(dfo.iloc[index]['humidity']):
                yp=MLPred('h', df2, spatialh, dfo.iloc[index]['ID'], ID2Index, dfo.iloc[index]['temperature'], index, extent1, extent2, extent3, extent4, extent5, w1, w2, w3, w4, w5, polydeg, risk_appetite)
                new_row = pd.Series([dfo.iloc[index]['ID'], yp], index=pred.columns)
                pred = pred.append(new_row,ignore_index=True)
                df2['timestamp']=df['timestamp']
            
for k in tqdm([1,2,5,7,8]):
    for l in tqdm([*range(1,9)], leave=False):
        dfo=pd.read_csv(path+'/TestSO_N50'+str(k)+'_'+str(l).zfill(2)+'.csv')
        df=pd.read_csv(path+'/TestS_N50'+str(k)+'_'+str(l).zfill(2)+'.csv')
        df.reset_index(drop=True, inplace=True)
        dfo.reset_index(drop=True, inplace=True)
        df2=df.copy()
        for index in tqdm(range(dfo.shape[0]), leave=False):
            if np.isnan(dfo.iloc[index]['temperature']):
                yp=MLPred('t', df2, testt, dfo.iloc[index]['ID'], TestID2Index, dfo.iloc[index]['humidity'], index, extent1, extent2, extent3, extent4, extent5, w1, w2, w3, w4, w5, polydeg, risk_appetite)
                new_row = pd.Series([dfo.iloc[index]['ID'], yp], index=test.columns)
                test = test.append(new_row,ignore_index=True)
                df2['timestamp']=df['timestamp']
            
            if np.isnan(dfo.iloc[index]['humidity']):
                yp=MLPred('h', df2, testh, dfo.iloc[index]['ID'], TestID2Index, dfo.iloc[index]['temperature'], index, extent1, extent2, extent3, extent4, extent5, w1, w2, w3, w4, w5, polydeg, risk_appetite)
                new_row = pd.Series([dfo.iloc[index]['ID'], yp], index=test.columns)
                test = test.append(new_row,ignore_index=True)
                df2['timestamp']=df['timestamp']
            
pred = pred.astype({'ID':'int32','Predictions':'float64'})    
test = test.astype({'ID':'int32','Predictions':'float64'})

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

In [6]:
#Check RMSE
test['Actual']=testval['Actual']
test['SquareDiff']=(test['Actual']-test['Predictions'])**2
RMSE = (test['SquareDiff'].mean())**0.5
print(RMSE)
sorted_test=test.sort_values(by=['SquareDiff'], ascending=False)
sorted_test

0.7843312190160858


,ID,Predictions,Actual,SquareDiff
44,759,26.351346,32.527,3.813871e+01
251,4252,45.126190,50.340,2.718382e+01
60,1012,46.506014,43.257,1.055609e+01
250,4231,39.172362,42.140,8.806878e+00
94,1619,32.417524,35.200,7.742173e+00
...,...,...,...,...
341,5656,12.245922,12.250,1.663241e-05
319,5330,60.124017,60.120,1.613845e-05
226,3781,17.739305,17.740,4.834070e-07
89,1509,23.220000,23.220,3.337348e-20


In [7]:
pred.to_csv('D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/pred.csv',index=False)
sorted_test.to_csv('D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/test.csv',index=False)